# Alguns conceitos de estatística, part III

 ## 1. Distribuição de Poisson
   - Mede a probabilidade de um evento ocorrer em intervalos de tempo, por exemplo: as chances de acidentes de carro em um dia
   - Os eventos a cada intervalo de tempo devem ser independentes entre si
   - Eventos:
     - P(X = x), exemplo: as chances de ocorrer 3 acidentes de carro no dia
     - P(X < x), exemplo: as chances de ocorrer menos de 3 acidentes no dia
     - P(X > x), exemplo: as chances de ocorrer mais de 3 acidentes no dia
   - Para o calculo é necessario:
     - O número de eventos que estão sendo calculados
     - O número médio de de ocorrência do evento no intervalo de tempo

In [4]:
import pandas as pandas
import numpy as numpy
import matplotlib.pyplot  as plt
from scipy import stats
from scipy.stats import poisson
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison
from pyod.models.knn import KNN

In [5]:
# Média de acidentes de carro é 2 por dia
# Qual a probabilidade de ocorrer 3 acidentes por dia?
poisson.pmf(3,2)

0.18044704431548356

In [6]:
# Qual a probabilidade de ocorrer 3 acidentes ou menos por dia?
poisson.cdf(3,2)

0.857123460498547

In [7]:
# Qual a probabilidade de ocorrer 3 acidentes ou mais por dia?
poisson.sf(3,2)

0.14287653950145296

 ## 2. Qui quadrado
  - Serve para a provar hipóteses